# Exploratory Data Analysis

# Data representation
1. **Corpus**: a collection of text
2. **Document-Term Matrix**: frequency matrix format

### Corpus

In [ ]:
# Read in the document-term matrix
import pandas as pd

data_df = pd.read_csv("lyrics.csv",index_col=0)
data_df

### Document-Term Matrix

In [ ]:
# Read in the document-term matrix
import pandas as pd

data_clean = pd.read_csv("lyrics_clean.csv",index_col=0)
data_clean

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

help(CountVectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())

vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1, 2),min_df=2, max_df=.8)#,stop_words=['the','this'] )
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names())
print(X2.toarray())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.lyrics)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
data_dtm[['love']]

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data_clean.lyrics)
len(vectorizer.get_feature_names())

In [ ]:
data_tfidf = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())
data_tfidf.index = data_clean.index
data_tfidf

In [ ]:
data_tfidf[['love']]

## Most Common Words

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} wordcloud

In [ ]:
# Find the top 30 words 

data=data_dtm.transpose()

top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

In [ ]:
# Print the top 30 words said by each artist
for artist, top_words in top_dict.items():
    print(artist)
    print(', '.join([word for word, count in top_words[0:30]]))
    print('---')

In [ ]:
from collections import Counter

# Let's first pull out the top 30 words for each artist
words = []
for artist in data.columns:
    top = [word for (word, count) in top_dict[artist]]
    for t in top:
        words.append(t)

In [ ]:
Counter(words).most_common()

In [ ]:
# If more than half of the comedians have it as a top word, exclude it from the list
add_stop_words = [word for word, count in Counter(words).most_common() if count > 8]
add_stop_words

In [ ]:
# Let's update our document-term matrix with the new list of stop words
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

stop_words

# Recreate document-term matrix
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.lyrics)
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = data_clean.index


In [ ]:
stop_words2=[]
for w in stop_words:
    stop_words2.append(w)

#stop_words2.extend(['girl','bird','la','li','ooh','let','number','jude'])

In [ ]:
# Let's make some word clouds!
# Terminal / Anaconda Prompt: conda install -c conda-forge wordcloud

#import sys
#!conda install --yes --prefix {sys.prefix} wordcloud

from wordcloud import WordCloud

wc = WordCloud(collocations=False,stopwords=stop_words2, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [ ]:
# Reset the output dimensions
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 10]


# Create subplots for each comedian
for index, artist in enumerate(data.columns):
    wc.generate(data_clean.lyrics[artist])
    
    plt.subplot(4, 4, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(data.columns[index])
    
plt.show()


## Number of Words

In [ ]:
# Identify the non-zero items in the document-term matrix, meaning that the word occurs at least once
unique_list = []
for artist in data.columns:
    uniques = data[artist].to_numpy().nonzero()[0].size
    unique_list.append(uniques)

# Create a new dataframe that contains this unique word count
data_words = pd.DataFrame(list(zip(data.columns, unique_list)), columns=['artist', 'unique_words'])
data_unique_sort = data_words.sort_values(by='unique_words')
data_unique_sort